## OpenQASM Parser

Implements a parser that generates an OpenQASM code that is equivalent to the input Amazon Braket quantum circuit

In [85]:
from braket.circuits import Circuit

In [86]:
QASMType = str

In [112]:
def circuit_to_qasm(circuit: Circuit) -> QASMType:
    """Converts a `braket.circuits.Circuit` to an OpenQASM string.
    Args:
        circuit: Amazon Braket quantum circuit

    Returns:
        The OpenQASM string equivalent to the circuit
    """
    
    # A mapping from Amazon Braket gates to QASM gates
    gates = {'cnot':'cx','ccnot':'ccx','i':'id','phaseshift':'p','si':'sdg','ti':'tdg', 'v':'sx', 'vi':'sxdg'}
    
    # Including the header
    code = "OPENQASM 2.0;\ninclude \"qelib1.inc\";\n\n"
    
    # Initializing the quantum register
    code += "qreg q[" + str(circuit.qubit_count) + "];\n\n"
    
    circuit_instr = circuit.instructions
    # Building the QASM codelines by applying gates one at a time
    for ins in circuit_instr:
        
        # Appending the gate name
        if ins.operator.name.lower() not in gates:
            code += ins.operator.name.lower()
        else:
            code += gates[ins.operator.name.lower()]
            
        # Appending parameters, if any
        try:
            param = "(" + str(ins.operator.angle) + ") "
        except:
            param = " "
        code += param
        
        # Appending the gate targets
        targets = [int(q) for q in ins.target]
        code += "q[{}]".format(targets[0])
        for t in range(1, len(targets)):
            code += ", q[{}]".format(targets[t])
        code += ";\n"

    return code

In [107]:
ghz_circ = Circuit()

ghz_circ.h(0)
ghz_circ.cnot(0, 1)
ghz_circ.cnot(1, 2)

print(ghz_circ)

T  : |0|1|2|
            
q0 : -H-C---
        |   
q1 : ---X-C-
          | 
q2 : -----X-

T  : |0|1|2|


In [108]:
print(circuit_to_qasm(ghz_circ))

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];

h q[0];
cx q[0], q[1];
cx q[1], q[2];



In [109]:
fourteen_circ = Circuit()

fourteen_circ.h([0, 1, 2, 3])
fourteen_circ.x([0, 1])
fourteen_circ.y(2)
fourteen_circ.z(3)
fourteen_circ.s(0)
fourteen_circ.si(1)
fourteen_circ.t(2)
fourteen_circ.ti(3)
fourteen_circ.rx(0, np.pi / 4)
fourteen_circ.ry(1, np.pi / 2)
fourteen_circ.rz(2, 3 * np.pi / 4)
fourteen_circ.v(0)
fourteen_circ.vi(1)
fourteen_circ.cnot(0, 1)

print(fourteen_circ)

T  : |0|1|2 |   3    |4 |5|
                           
q0 : -H-X-S--Rx(0.79)-V--C-
                         | 
q1 : -H-X-Si-Ry(1.57)-Vi-X-
                           
q2 : -H-Y-T--Rz(2.36)------
                           
q3 : -H-Z-Ti---------------

T  : |0|1|2 |   3    |4 |5|


In [110]:
print(circuit_to_qasm(fourteen_circ))

OPENQASM 2.0;
include "qelib1.inc";

qreg q[4];

h q[0];
h q[1];
h q[2];
h q[3];
x q[0];
x q[1];
y q[2];
z q[3];
s q[0];
sdg q[1];
t q[2];
tdg q[3];
rx(0.7853981633974483) q[0];
ry(1.5707963267948966) q[1];
rz(2.356194490192345) q[2];
sx q[0];
sxdg q[1];
cx q[0], q[1];

